# COMP 551 final project code
paper: Unleashing the Potential of CNNs for Interpretable Few-Shot Learning

## data loading 
meta training set: 64 classes - training CNN
meta testing set: 20 classes - extract VC and validate
meta validation set: 16 classes - Not being used

On each trial (5-shot)
- Use 25 samples(5 classes) from training split to train CNN to achieve high accuracy.
- Use 25 samples(5 classes) from train subset of the testing split to extract VC
- Use 15 samples for each class(5 classes) form test subset of the testing split to classify images using VC.

10 trials:

desired output file: 

training set : (64 classes in total)
- train_x_1.csv : 5 sample each of 5 classes , train_y_1.csv : 5 sample for each of 5 classes
- test_x_1.csv : 5 sample each of 3 classes, test_y_1.csv : 5 sample each of 3 classes

- train_x_2.csv.....
- test_x_2.csv
....
....
- train_x_10.csv.....
- test_x_10.csv

testing set : (20 classes in total)
- train_x_1.csv : 5 sample each of 5 classes , train_y_1.csv : 5 sample for each of 5 classes
- test_x_1.csv : 5 sample each of 3 classes, test_y_1.csv : 5 sample each of 3 classes

- train_x_2.csv.....
- test_x_2.csv
....
....
- train_x_10.csv.....
- test_x_10.csv

the mini-imagenet structure
```
paper_reproduce_dataset/train/n01882714/images/n01882714_0.jpeg
```

use load_labels() to controll the number of class uses in a trial
then use load_data to get train_x, train_y



In [1]:
#helper function to import data
import re

def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)
    return l

#load data
import skimage
from skimage.io import ImageCollection,concatenate_images,imread

from skimage.color import gray2rgb
import numpy as np
from skimage import io
from os import listdir
from os.path import isfile, join


def print_image(data):
    from matplotlib import pyplot as plt
    plt.imshow(data, interpolation='nearest')
    plt.show()

def load_labels():
    labels = []
    with open("paper_reproduce_dataset/wnids.txt","r") as input:
        for row in input:
            labels.append(row[0:9])
    return labels

def imreadconvert(Xname):
    
    X=imread(Xname)
    if len(X.shape)==3:
        return X
    else:
        return gray2rgb(X)  
    
def load_data(label_dict,dataset):
    print ("loading",dataset, "data!")
    nsamples=10 #5 images for each of 200 labels
    file_names=[]
    labels=[] 
    
    for label in label_dict:
        #print str(label) 
        cur_dir="paper_reproduce_dataset/"+label+"/images"
        onlyfiles = [f for f in sort_nicely(listdir(cur_dir)) if isfile(join(cur_dir, f))]
        onlyfiles = random.sample(onlyfiles,nsamples)
        onlyfiles=[cur_dir+'/'+f for f in onlyfiles]
        file_names=file_names+onlyfiles    
        cur_labels=nsamples*[label]
        labels=labels+cur_labels
    image_collect = ImageCollection(file_names,load_func=imreadconvert)
    x_data = concatenate_images(image_collect)   
    print ("loaded",dataset, "data")
    y_data=np.asarray(labels)
    y_data=np.reshape(y_data,(len(y_data),))
   
    print("x_",dataset,".shape =",x_data.shape)
    print("y_",dataset,".shape =",y_data.shape)
   
    return x_data,y_data 
        

    

In [2]:
#VGG model structure
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D , AveragePooling2D,ZeroPadding2D


def VGG_13(num_class):
    model = Sequential()
    
    
    model.add(Conv2D(64, (3, 3), activation="relu",padding="same",input_shape=(64,64,3)))
    model.add(Conv2D(64, (3, 3), activation="relu",padding="same"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Conv2D(128, (3, 3), activation="relu",padding="same"))
    model.add(Conv2D(128, (3, 3), activation="relu",padding="same"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Conv2D(256, (3, 3), activation="relu",padding="same"))
    model.add(Conv2D(256, (3, 3), activation="relu",padding="same"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Conv2D(512, (3, 3), activation="relu",padding="same"))
    model.add(Conv2D(512, (3, 3), activation="relu",padding="same"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Conv2D(512, (3, 3), activation="relu",padding="same"))
    model.add(Conv2D(512, (3, 3), activation="relu",padding="same"))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_class, activation='softmax'))

#     if weights_path:
#         model.load_weights(weights_path)

    return model



Using TensorFlow backend.


In [3]:
#data split and train model function
import random

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer

from keras import optimizers

def save_model(model_to_save, json_name,h5_name):
#     serialize model to JSON
    model_json = model_to_save.to_json()
    with open(json_name, "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model_to_save.save_weights(h5_name)
    print("Saved model to disk")

    
from keras.models import model_from_json  

def load_model(json_name,h5_name):
    # load json and create model
    json_file = open(json_name, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(h5_name)
    print("Loaded model from disk")
    return loaded_model


def split_metadata():
    label_dict=load_labels()
    train_split_label = label_dict[0:64]
    test_split_label = label_dict[64:80]
    
    #spliting dataset to 64 training class with 10 samples per class
    #and 16 testing class with 10 samples per class
    x_train,y_train = load_data(train_split_label,"train")
    print()
    x_test,y_test = load_data(test_split_label,"test")
    return x_train,y_train,x_test,y_test

def run_CNN_trial(train_trial):
    #set hyper param
    epochs = 30
    learning_rate = 0.01
    
    train_trial = list(zip(*train_trial))
    x_tr = np.asarray(train_trial[0])
    y_tr = np.asarray(train_trial[1])
    
    #preprocessing training data
    lb = preprocessing.LabelBinarizer()
    onehot_y_tr = lb.fit_transform(y_tr)
    num_class = onehot_y_tr.shape[1]
    x_train_tr,x_test_tr,y_train_tr,y_test_tr = train_test_split(x_tr, onehot_y_tr, test_size=0.5, random_state=42)
    
    cnn_model = VGG_13(num_class)
    print("start training CNN model.")
    
    optimizer = optimizers.SGD(lr=learning_rate)
    cnn_model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])

    from keras.callbacks import ModelCheckpoint

    checkpoint = ModelCheckpoint("best_model.hdf5", monitor='acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    
    cnn_model.fit(x_train_tr, y_train_tr,
              epochs=epochs,
              callbacks=callbacks_list,    
              verbose=1,
              validation_data=(x_test_tr, y_test_tr))

    score = cnn_model.evaluate(x_test_tr, y_test_tr, verbose=0)
    
#     print('Test loss:', score[0])
#     print('Test accuracy:', score[1])
    
#     print("finish training CNN model.")
    
    return loaded_model

def train_model():
    
    x_train,y_train,x_test,y_test = split_metadata()
   
    train_data = list(zip(x_train,y_train))
    test_data = list(zip(x_test,y_test))
    #random choose 50 samples from training split to train CNN for one trail
    train_trial = random.sample(train_data,50)
    
#     cur_image = "paper_reproduce_dataset/"+"n01443537/images/"+"n01443537_0.JPEG"
#     print_image(imreadconvert(cur_image))
    
#     print_image(train_trial[0][0])
#     print(train_trial[0][1])
#     print_image(train_trial[1][0])
#     print(train_trial[1][1])
#     print_image(train_trial[2][0])
#     print(train_trial[2][1])
    
    cnn_model = run_CNN_trial(train_trial)
    return x_test,y_test,cnn_model 


In [4]:
# K-mean clustering visual concept
from sklearn.cluster import KMeans

from keras import backend as K

#get intermidiate layer output from model
#3rd maxpooling output
def get_int_layer(model,input):
    get_3rd_layer_output = K.function([model.layers[0].input],
                                      [model.layers[8].output])
    layer_output = get_3rd_layer_output([input])[0]
    return layer_output

def extract_VC(layer_output):
    num_sample =  layer_output.shape[1] * layer_output.shape[2]
    input_kmean = layer_output.reshape(num_sample,layer_output.shape[0] * layer_output.shape[3])
    kmeans = KMeans(n_clusters=20,random_state=0).fit(input_kmean)
#     print(kmeans.labels_)
    visual_concepts = kmeans.cluster_centers_
    
    return visual_concepts

def get_VC_dict(cnn_model,x_test,y_test):
    test_data_dict_x=dict()
    test_data_dict_y=dict()
    vc_dict=dict()
    for row in range(x_test.shape[0]):
        if y_test[row] not in test_data_dict_x:
            test_data_dict_x[y_test[row]]=[x_test[row]]
        else:
            test_data_dict_x[y_test[row]].append(x_test[row])
    #for each class use half data to extract VC, half to validate
    test_data_dict_train_x={key:0 for key in test_data_dict_x.keys()}
    
    test_data_dict_valid_x={key:0 for key in test_data_dict_x.keys()}
#     print(test_data_dict_x)
    for key in test_data_dict_x.keys():
        half_len = int(len(test_data_dict_x[key])/2)
        test_data_dict_train_x[key]=test_data_dict_x[key][0:half_len]
        test_data_dict_valid_x[key]=test_data_dict_x[key][half_len::]
    
    vc_dict={key:0 for key in test_data_dict_x.keys()}

    for key in test_data_dict_x.keys():
        layer_output = get_int_layer(cnn_model,test_data_dict_train_x[key])
        vc_dict[key]=extract_VC(layer_output)
        
    return vc_dict,test_data_dict_train_x,test_data_dict_valid_x
# for el in list(test_data_dict_valid_y.values()):
#     print(el)

In [31]:
#distance function for each pixel p in intermediate layer and one Visual Concept
#input shape : f_p:(256,) f_vc:(256,)
from numpy import linalg as LA
def d_p_v(f_p,f_vc):
    f_p=f_p.astype(np.float64)
    f_vc=f_vc.astype(np.float64)
    
    fp_fvc = np.dot(f_p,f_vc)
    norm_fp_fvc = LA.norm(f_p)*LA.norm(f_vc)
    distance = 1- fp_fvc/norm_fp_fvc
    
    return distance

# print(list(vc_dict.values())[0][0])
# print(distance_vc(a,list(vc_dict.values())[0][0]))

#VC-encoding
def thresholding(f_p,f_vc):
    
    f_vc=f_vc.reshape(5,256)
    f_vc = np.mean(f_vc,0)
    threshold = 0.002
    distance_vc = d_p_v(f_p,f_vc)
    print(distance_vc)
    if distance_vc<threshold:
        return 1
    else:
        return 0
    
    
#Nearest Neighbor similarity
#layer_output is in shape of 64*64

def manhattan_neighbours(x, y, w, h, distance):
    for i in range(x-distance, x+1+distance):
        for j in range(y-distance, y+1+distance):
            if i < 0 or j < 0:
                continue
            if i >= w or j >= h:
                continue
            if i == x and j == y:
                continue
            yield (i, j)
        

def inverse_similarity(input_image,train_image):
#     print("start")
    global vc_dict, loaded_model
    input_image = input_image.reshape(1,64,64,3)
    train_image = train_image.reshape(1,64,64,3)

    ps = get_int_layer(loaded_model, input_image)[0]  # b
    pps = get_int_layer(loaded_model, train_image)[0]  # b prime
    x, y = ps.shape[:2]

    vcs = []
    for li in vc_dict.values():
        vcs.extend(li)

    num1 = 0
    denum1 = 0
    num2 = 0
    denum2 = 0
    for vc in vcs:
        for i in range(x):
            for j in range(y):
                b1 = thresholding(ps[i,j,:], vc)
                b2 = thresholding(pps[i,j,:], vc)
                qmax1 = max(thresholding(pps[i2,j2,:], vc) for i2, j2 in manhattan_neighbours(i, j, x, y, 2))
                qmax2 = max(thresholding(ps[i2,j2,:], vc) for i2, j2 in manhattan_neighbours(i, j, x, y, 2))
                num1 += b1 * qmax1
                denum1 += b1
                num2 += b2 * qmax2
                denum2 += b2 
    
    similarity = 0.5 * (num1/denum1 + num2/denum2)
    return 1/similarity
from sklearn.neighbors import KNeighborsClassifier

def nearest_neighbor(cnn_model,test_data_dict_valid_x,test_data_dict_train_x,vc_dict):
    train_x = []
    train_y = []
    test_x =[]
    test_y =[]
    image_classes = list(vc_dict.keys())
    for image_class in image_classes:
        for data_x in test_data_dict_train_x[image_class]:
            train_x.append(data_x)
            train_y.append(image_class)
        for data_x in test_data_dict_valid_x[image_class]:
            test_x.append(data_x)
            test_y.append(image_class)
    print(len(train_x))
    train_x = train_x[0:15]
    train_y = train_y[0:15]
    test_x = test_x[0:15]
    test_y = test_y[0:15]
    
    print(train_y,test_y)
    
    nbrs = KNeighborsClassifier(n_neighbors=4, algorithm='ball_tree', metric=inverse_similarity)
    for i in range(len(train_x)):
        el = train_x[i].reshape(train_x[i].shape[0]*train_x[i].shape[1]*train_x[i].shape[2])
        train_x[i]=el
    for i in range(len(test_x)):
        el = test_x[i].reshape(test_x[i].shape[0]*test_x[i].shape[1]*test_x[i].shape[2])
        test_x[i]=el
        
    nbrs.fit(train_x,train_y)
    
    return nbrs.score(test_x,test_y)
#     return nbrs.predict(input_image_x.reshape(1,12288))

# def total_accuracy(cnn_model,vc_dict,test_data_dict_valid_x,test_data_dict_train_x):
#     image_classes = list(vc_dict.keys())
#     true_positive = 0
#     total = 0
#     for image_class in image_classes:
#         image_list = test_data_dict_valid_x[image_class]
#         for image_id in range(len(image_list)):
#             pred_class = nearest_neighbor(cnn_model,image_list[image_id],test_data_dict_train_x,vc_dict)
#             if pred_class == image_class:
#                 true_positive += 1
#             total += 1
#     return true_positive/total

In [6]:
#train model
# x_test,y_test,cnn_model = train_model()  
# save_model(cnn_model, "VC_CNN.json", "VC_CNN.hdf5")

loaded_model = load_model("VC_CNN.json","best_model.hdf5")
x_train,y_train,x_test,y_test = split_metadata()


Loaded model from disk
loading train data!
loaded train data
x_ train .shape = (640, 64, 64, 3)
y_ train .shape = (640,)

loading test data!
loaded test data
x_ test .shape = (160, 64, 64, 3)
y_ test .shape = (160,)


In [7]:
#Obtain VC dictionary and test dictionary
vc_dict,test_data_dict_train_x,test_data_dict_valid_x=get_VC_dict(loaded_model,x_test,y_test)


In [32]:
#print result accuracy for this trial
# accuracy = total_accuracy(loaded_model,vc_dict,test_data_dict_valid_x,test_data_dict_train_x)
accuracy = nearest_neighbor(loaded_model,test_data_dict_valid_x,test_data_dict_train_x,vc_dict)
print(accuracy)

80
['n03796401', 'n03796401', 'n03796401', 'n03796401', 'n03796401', 'n02795169', 'n02795169', 'n02795169', 'n02795169', 'n02795169', 'n02123045', 'n02123045', 'n02123045', 'n02123045', 'n02123045'] ['n03796401', 'n03796401', 'n03796401', 'n03796401', 'n03796401', 'n02795169', 'n02795169', 'n02795169', 'n02795169', 'n02795169', 'n02123045', 'n02123045', 'n02123045', 'n02123045', 'n02123045']
0.00830370764221
0.00851531141822
0.00232902021282
0.00229740079791
0.00710639300772
0.00225453052347
0.00223068703047
0.0075221388242
0.00378456901019
0.00320180266274
0.00206521662678
0.00202033587725
0.00722889855441
0.00242860480831
0.00250513846952
0.00772803862528
0.0032756052705
0.0032417919566
0.00206521662678
0.00232902021282
0.00851531141822
0.00229740079791
0.00228488263912
0.00710639300772
0.00225453052347
0.00223068703047
0.00212921822087
0.0075221388242
0.00378456901019
0.00320180266274
0.00297605010283
0.00830370764221
0.00202033587725
0.00196823350055
0.00722889855441
0.002428604808

0.00228488263912
0.00214917859179
0.00213672566612
0.00226788239897
0.00223068703047
0.00212921822087
0.00196991459368
0.0019748736096
0.00196983977118
0.00320180266274
0.00297605010283
0.00365746295195
0.00307177481174
0.00314668252162
0.00301553550334
0.00281454127779
0.00221412270535
0.00302196976616
0.00362875559927
0.00314649866387
0.00236263695609
0.00208003659384
0.00310217105499
0.00202033587725
0.00196823350055
0.00177618023155
0.00176741250337
0.00203293088787
0.00250513846952
0.00255308785568
0.00237249412258
0.00222193323726
0.0024780813444
0.0032417919566
0.00305432631512
0.00245808355553
0.00271252490406
0.00244007796687
0.00252538708152
0.00255844227926
0.00238709869675
0.00209533797404
0.00264694129249
0.00267428019022
0.0027014895142
0.00255078907615
0.00204068040939
0.00245808355553
0.00365746295195
0.00228488263912
0.00214917859179
0.00213672566612
0.00226788239897
0.00616538706756
0.00212921822087
0.00196991459368
0.0019748736096
0.00196983977118
0.00472354481579
0.

0.0119839681026
0.00469028863785
0.00595914481562
0.00221412270535
0.00302196976616
0.00488340442589
0.00208003659384
0.00310217105499
0.00501733940025
0.00380456541699
0.00449061373354
0.00213163708814
0.00233731728473
0.00482353852891
0.00726470943109
0.00742871943819
0.0111407224606
0.00238709869675
0.00209533797404
0.00502203926833
0.00255078907615
0.00204068040939
0.00447724461758
0.00264805403698
0.00223157748574
0.00236224718663
0.00212839189908
0.00477060956096
0.00869530887335
0.00862017521052
0.0119839681026
0.0064123602096
0.0065209094846
0.00605377157315
0.00221125597301
0.00362875559927
0.00605288000348
0.00130848888791
0.00203135357935
0.00141583019613
0.00118221060891
0.0143732067213
0.00844242036966
0.0075622489648
0.00733881134744
0.00279266351978
0.00264694129249
0.00691019980949
0.00286085121098
0.00341503606146
0.00172546069925
0.00196194742782
0.0140755486269
0.0087370060025
0.00882177118671
0.00172546069925
0.00141583019613
0.00605377157315
0.00221125597301
0.0036

0.000168977229856
0.000284087704783
0.000178017179904
0.000305251192683
0.000288835859848
0.00054622068091
0.000644775045498
0.00029195917144
0.000256516639188
0.000364632118941
0.00022605071337
0.000487687377916
0.000546007630648
0.000398339598509
0.00017467404538
0.000187750295447
0.000512766424669
0.000833873918531
0.000779744339084
0.000703209117062
0.000633256823389
0.00093316386101
0.000222473564202
0.000209025041909
0.000170425233385
0.000234432667518
0.000214277291552
0.000351348172184
0.00028025028309
0.000214364545128
0.000239452935607
0.000256998329388
0.000205454146241
0.000200293281466
0.000203640505845
0.000174271330267
0.000200730588958
0.000224585527471
0.000247984350363
0.000236489368412
0.000197403990193
0.000417851902399
0.000284206305739
0.00027259389223
0.000341404189411
0.000170845220267
0.000203640505845
0.00022605071337
0.000185089974938
0.000168977229856
0.000284087704783
0.000178017179904
0.0051551329173
0.000288835859848
0.00054622068091
0.000644775045498
0.0

0.00618006713885
0.00619101433207
0.00620285054756
0.00268903607638
0.000817735703505
0.00090451055477
0.000934511235529
0.00251247549086
0.00115383659666
0.000914094268568
0.00104375068548
0.00140752574196
0.00226207629778
0.0010942276503
0.000974987113194
0.00132175458533
0.00125952030542
0.00813066941422
0.00568033353761
0.00570369042558
0.00562195941049
0.00570000388343
0.00266676480228
0.000829444812525
0.000885128723672
0.000958439998975
0.00261052487288
0.00107411370043
0.00112891773044
0.00106075042761
0.00106856518976
0.00254955267134
0.000965676501327
0.000980225924201
0.000988276772
0.00103374625818
0.000885128723672
0.00090451055477
0.0060882535223
0.00618006713885
0.00619101433207
0.00620285054756
0.00657396434605
0.000817735703505
0.000856532905151
0.000934511235529
0.00108728656735
0.00115383659666
0.000914094268568
0.00104375068548
0.00140752574196
0.0014781818396
0.0010942276503
0.000974987113194
0.00132175458533
0.00125952030542
0.00109647156941
0.00568033353761
0.005

...

0.00141488145127
0.0015808821379
0.00119806754425
0.00776465224863
0.00167425454804
0.00179475815193
0.00157398645979
0.00837338204535
0.0118503584351
0.0133522345516
0.00566185470539
0.00567741951205
0.00133487846964
0.00112545653637
0.00814666024454
0.0021756092689
0.00143494632545
0.00795757263003
0.00505464314933
0.00465184412295
0.0015808821379
0.00119806754425
0.00776465224863
0.00179475815193
0.00157398645979
0.00837338204535
0.00285819246588
0.00281485680447
0.0075746665521
0.00511062610542
0.00520755924211
0.00100652580497
0.00111164873646
0.00305256484495
0.00163274699276
0.00139661726135
0.00282191272244
0.00150201217794
0.00165690218333
0.00720934560473
0.0046874401853
0.00469552769794
0.00112979324161
0.00120909702862
0.00311544307793
0.00167882194623
0.00173725148051
0.00303623336971
0.00143623443421
0.00142520252387
0.00112979324161
0.00100652580497
0.0075746665521
0.00511062610542
0.00520755924211
0.00522292519607
0.00281485680447
0.00111164873646
0.00118458223466
0.003

0.00139661726135
0.00157574528944
0.0020642986815
0.0021756092689
0.00143494632545
0.00165690218333
0.00213475377549
0.00197195386864
0.00159344680165
0.00125478299024
0.00165085296706
0.00138895998492
0.00132541065506
0.00119054331862
0.00267249190941
0.00194648864658
0.00181015091835
0.00172383727432
0.00144355918378
0.00243373607029
0.00283165572194
0.00318215873814
0.0032130363195
0.00241059276557
0.00173725148051
0.00165101976399
0.00167425454804
0.00179475815193
0.00157398645979
0.00142520252387
0.00145304551369
0.00155797095908
0.00152990983126
0.00148982071909
0.00128626749615
0.00147383914654
0.00130468054991
0.00133509656104
0.00183192171448
0.00161806809591
0.00135891456173
0.00123220851562
0.00127865838253
0.0012989931419
0.0012842258616
0.0013176361247
0.00130050246224
0.00130231416587
0.00130468054991
0.00132541065506
0.00157574528944
0.0020642986815
0.0021756092689
0.00143494632545
0.00795757263003
0.00213475377549
0.00197195386864
0.00159344680165
0.00125478299024
0.007

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




0.00330387298355
0.00332247407872
0.0033335171018
0.0108294549134
0.00161806809591
0.00135891456173
0.00123220851562
0.00127865838253
0.00813453612608
0.0012842258616
0.0013176361247
0.00130050246224
0.00130231416587
0.00820833455387
0.0038503283848
0.00375002016492
0.00392874275502
0.0114222739547
0.00392874275502
0.0033335171018
0.00181015091835
0.00172383727432
0.00144355918378
0.00760282515975
0.00318215873814
0.0032130363195
0.00241059276557
0.00852565370541
0.00332247407872
0.00334364574708
0.0108294549134
0.00135891456173
0.00123220851562
0.00127865838253
0.00813453612608
0.0013176361247
0.00130050246224
0.00130231416587
0.00820833455387
0.00375002016492
0.00371314397078
0.0114222739547
0.0114222739547
0.0108294549134
0.00172383727432
0.00144355918378
0.00760282515975
0.0032130363195
0.00241059276557
0.00852565370541
0.00334364574708
0.0033335171018
0.00123220851562
0.00127865838253
0.00813453612608
0.00130050246224
0.00130231416587
0.00820833455387
0.00371314397078
0.003928742

KeyboardInterrupt: 